In [4]:
import logging

import pandas as pd

from shmple import AttentionModel

from epam.mutsel_simulation import parent_position_is_hydrophobic, MutSelSimulator
from epam.toy_simulation import mimic_mutations

In [2]:
shmple_weights_directory = "/Users/matsen/re/epam/data/shmple_weights/my_shmoof"
shmple_model = AttentionModel(weights_dir=shmple_weights_directory, log_level=logging.WARNING)

simulator = MutSelSimulator(parent_position_is_hydrophobic, shmple_model)
parent_dna_sequence = "GCTGTCGCAAAGCGT"
target_mut_count = 5
child_dna_sequence = simulator.simulate_child_sequence(parent_dna_sequence, target_mut_count)
print("Simulated child DNA sequence:", child_dna_sequence)


Simulated child DNA sequence: CGT


In [6]:
pcp_df = pd.read_csv("~/data/wyatt-10x-1p5m_pcp_2023-10-07.csv", index_col=0)
pcp_df = pcp_df[pcp_df["parent"] != pcp_df["child"]].reset_index(drop=True)
pcp_df["sub_count"] = pcp_df.apply(lambda row: sum([p != c for p, c in zip(row["parent"], row["child"])]), axis=1)
pcp_df

,sample_id,family,parent,child,v_gene,child_is_leaf,sub_count
0,0,149198,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGA...,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGA...,IGHV3-33*01,False,3
1,0,149198,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGA...,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGA...,IGHV3-33*01,False,4
2,0,149198,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGA...,CAAGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGA...,IGHV3-33*01,False,16
3,0,149198,CAAGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGA...,CAAATGCAGATGGTGGAGTCGGGGGGAGGCGTGGTCCAGCCAGGGA...,IGHV3-33*01,True,19
4,0,149198,CAAGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGA...,CAAGTGCAACTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGA...,IGHV3-33*01,True,7
...,...,...,...,...,...,...,...
60288,0,17934,CAGCTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGG...,CAGCTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGG...,IGHV4-39*01,True,17
60289,0,17934,CAGCTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGG...,CAGCTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGG...,IGHV4-39*01,False,2
60290,0,17934,CAGCTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGG...,CAGCTGCAGCTGCAGGAGTCGGGCCCAGGACTTGTGAAGCCTTCGG...,IGHV4-39*01,True,8
60291,0,269306,GAGGTGCAGCTGTTGGAGTCTGGGGGAGGCTTGGTACAGCCTGGGG...,GAGGTGCAGCTGTTGGAGTCTGGGGGAGGCTTGGTACAGCCTGGGG...,IGHV3-23*01,True,3


In [ ]:
fake_df = pd.DataFrame({"parent": pcp_df["parent"], "child": mimic_mutations(simulator.simulate_child_sequence, pcp_df["parent"], pcp_df["sub_count"])})

fake_df["sub_count"] = pcp_df.apply(lambda row: sum([p != c for p, c in zip(row["parent"], row["child"])]), axis=1)

assert (fake_df["sub_count"] == pcp_df["sub_count"]).all()